In [1]:
import json
import mysql.connector
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from urllib3.exceptions import InsecureRequestWarning
import urllib3

# PART I Data Process

In [2]:
# load the raw data to a dictionary

def load_data(file_name):

    lines = 0
    tweets = {}

    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                lines = lines + 1
                if data['id_str'] not in tweets:
                    tweets[data['id_str']] = data
            except:
                continue
    
    return tweets

In [37]:
tweets = load_data("corona-out-3")
print(tweets.__len__())

101894


In [38]:
parent_tweet = {}
for key in tweets:
    if 'retweeted_status' in list(tweets[key].keys()):
        record = tweets[key]['retweeted_status']
        id = record['id_str']
        parent_tweet[id] = record

print(parent_tweet.__len__())

14842


In [39]:
tweets.update(parent_tweet)
print(tweets.__len__())

112023


In [40]:
pro_tweets = {}
feature_list1 = ['id_str', 'text', 'created_at', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count']
feature_list2 = ['name', 'location', 'url']

for key in tweets:
    pro_tweets[key] = {}
    try:
        for feature in feature_list1:
            pro_tweets[key][feature] = tweets[key][feature]
        pro_tweets[key]['popularity'] = sum(pro_tweets[key][feature_list1[i]] for i in range(3, 7))
        pro_tweets[key]['user_id_str'] = tweets[key]['user']['id_str']
        for feature in feature_list2:
            pro_tweets[key][feature] = tweets[key]['user'][feature]
        pro_tweets[key]['user_popularity'] = 0
        pro_tweets[key]['retweeted'] = ''
        pro_tweets[key]['retweet'] = []
        pro_tweets[key]['retweeted'] += tweets[key]['retweeted_status']['id_str']
    except:
        continue

In [41]:
for key in pro_tweets:
    if pro_tweets[key]['retweeted']:
        try:
            pro_tweets[pro_tweets[key]['retweeted']]['retweet'].append(key)
        except:
            continue

In [64]:
beta_data = pro_tweets['1254022770679320576']
beta_data

{'id_str': '1254022770679320576',
 'text': 'É isto, ou vou morrer sem ar ou com o corona https://t.co/O0Y7B3Koj4',
 'created_at': 'Sat Apr 25 12:21:41 +0000 2020',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'popularity': 0,
 'user_id_str': '804046791348015107',
 'name': 'Bi Sex Uau',
 'location': 'Acre, Brasil',
 'url': 'https://www.instagram.com/?hl=pt-br',
 'user_popularity': 0,
 'retweeted': '',
 'retweet': []}

# PART II MySQL

In [44]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="emmerich0315",
  database="DatabaseFinal"
)

cursor = mydb.cursor()

In [45]:
# intialize

query_1 = "DROP TABLE user_information;"
query_2 = "DROP TABLE tweet_information;"
query_3 = "DROP TABLE tweet_popularity;"

cursor.execute(query_1)
cursor.execute(query_2)
cursor.execute(query_3)

In [46]:
sql_tweet_info = '''
CREATE TABLE tweet_information (
tweet_id VARCHAR(255) PRIMARY KEY, 
user_id VARCHAR(255),
created_at VARCHAR(255), 
retweet JSON,
retweeted VARCHAR(255)
)
'''
cursor.execute(sql_tweet_info)

In [47]:
sql_user_info = '''
CREATE TABLE user_information (
user_id VARCHAR(255) PRIMARY KEY, 
name VARCHAR(255), 
location VARCHAR(255),
url VARCHAR(255),
user_popularity INT
)
'''
cursor.execute(sql_user_info)

In [48]:
sql_tweet_popularity = '''
CREATE TABLE tweet_popularity (
tweet_id VARCHAR(255) PRIMARY KEY,
quote_count INT ,
reply_count INT ,
retweet_count INT ,
favourite_count INT ,
popularity INT
)
'''
cursor.execute(sql_tweet_popularity)

In [49]:
for key, val in pro_tweets.items():
    cursor = mydb.cursor()
    query = "INSERT INTO tweet_information (tweet_id, user_id, created_at, retweet, retweeted) VALUES (%s, %s, %s, %s, %s)"
    param = (val['id_str'], val['user_id_str'], val['created_at'], json.dumps(val['retweet']), val['retweeted'])
    cursor.execute(query, param)
    mydb.commit()

In [50]:
pd.read_sql('SELECT * FROM tweet_information limit 5', con=mydb)

,tweet_id,user_id,created_at,retweet,retweeted
0,1210899520692740096,1133262236745588737,Sat Dec 28 12:25:17 +0000 2019,"[""1254022996232028161""]",
1,1217113384291880962,61519237,Tue Jan 14 15:56:58 +0000 2020,"[""1254050769202032642""]",
2,1221138283675021312,29280466,Sat Jan 25 18:30:29 +0000 2020,"[""1254053174052360192""]",
3,1221341690142679041,349747452,Sun Jan 26 07:58:45 +0000 2020,"[""1254039171905150978"", ""1254039540383145990""]",
4,1222318351357054976,743662396892282881,Wed Jan 29 00:39:39 +0000 2020,"[""1254039762962460672""]",


In [51]:
for key, val in pro_tweets.items():
    cursor = mydb.cursor()
    query = "INSERT INTO user_information (user_id, name, location, url, user_popularity) VALUES (%s, %s, %s, %s, %s)"
    param = (val['user_id_str'], val['name'], val['location'], val['url'], val['user_popularity'])
    try:
        cursor.execute(query, param)
        mydb.commit()
    except:
        continue

In [52]:
pd.read_sql('SELECT * FROM user_information limit 5', con=mydb)

,user_id,name,location,url,user_popularity
0,1000006582896295938,sara,None,None,0
1,1000026406112251905,Corona supplier banker,Hell,None,0
2,1000027886915637250,cheche,None,None,0
3,1000034375973646337,clarih 🐻,Bonsucesso,None,0
4,100004211,azakiya tamilmagan,"Chennai, India",None,0


In [53]:
for key, val in pro_tweets.items():
    cursor = mydb.cursor()
    query = "INSERT INTO tweet_popularity (tweet_id, quote_count, reply_count, retweet_count, favourite_count, popularity) VALUES (%s, %s, %s, %s, %s, %s)"
    param = (val['id_str'], val['quote_count'], val['reply_count'], val['retweet_count'], val['favorite_count'], val['popularity'])
    cursor.execute(query, param)
    mydb.commit()

In [55]:
pd.read_sql('SELECT * FROM tweet_popularity limit 5', con=mydb)

,tweet_id,quote_count,reply_count,retweet_count,favourite_count,popularity
0,1210899520692740096,0,1,3,2,6
1,1217113384291880962,0,2,15,70,87
2,1221138283675021312,5,23,306,797,1131
3,1221341690142679041,68,83,292,286,729
4,1222318351357054976,1072,2719,11983,61466,77240


# PART III ES

In [57]:
urllib3.disable_warnings(InsecureRequestWarning)

es = Elasticsearch(["https://localhost:9200"], http_auth=('elastic', '1D+eJ1FMf71gUJHx85Mi') ,verify_certs=False)

es_info = es.info()

print(es_info)

/Users/liyunhao/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:394: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/ml/j_s90hn95cbf06qnzv38yz1m0000gn/T/ipykernel_6381/1593725772.py:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(["https://localhost:9200"], http_auth=('elastic', '1D+eJ1FMf71gUJHx85Mi') ,verify_certs=False)


{'name': 'liyunhaodeMBP', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'o7bctgOwRfCYSEKlL3KY9Q', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [58]:
if es.indices.exists(index='tweets'):
    es.indices.delete(index='tweets')

In [59]:
actions = [
    {
        "_index": 'tweets',
        "_id": key,
        "_source": {"text": val.get('text')},
    }
    for key, val in pro_tweets.items()
]
helpers.bulk(es, actions)

(112023, [])

In [60]:
response = es.get(index='tweets', id='1254022770679320576')
response['_source']['text']

'É isto, ou vou morrer sem ar ou com o corona https://t.co/O0Y7B3Koj4'

# PART IV Cache